In [1]:
import nltk; nltk.download('stopwords')
!python3 -m spacy download en
!pip install pyLDAvis
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.0 MB 8.4 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 9.8 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=1c88c5ed6a97ad1fb517f0d7076beec3b7

True

In [2]:

!wget http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
!unzip -q mallet-2.0.8.zip
!mallet-2.0.8/bin/mallet

--2022-06-10 23:30:32--  http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
Resolving mallet.cs.umass.edu (mallet.cs.umass.edu)... 128.119.246.70
Connecting to mallet.cs.umass.edu (mallet.cs.umass.edu)|128.119.246.70|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://mallet.cs.umass.edu/dist/mallet-2.0.8.zip [following]
--2022-06-10 23:30:33--  https://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
Connecting to mallet.cs.umass.edu (mallet.cs.umass.edu)|128.119.246.70|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16184794 (15M) [application/zip]
Saving to: ‘mallet-2.0.8.zip’

mallet-2.0.8.zip    100%[===================>]  15.43M  16.8MB/s    in 0.9s    

2022-06-10 23:30:34 (16.8 MB/s) - ‘mallet-2.0.8.zip’ saved [16184794/16184794]

Unrecognized command: 
Mallet 2.0 commands: 

  import-dir         load the contents of a directory into mallet instances (one per file)
  import-file        load a single file into mallet instances (

In [3]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy
nlp = spacy.load("en_core_web_sm")

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)


/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


In [4]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
#You should update this path as per the path of Mallet directory on your system.
mallet_path = 'mallet-2.0.8/bin/mallet'

In [5]:
df = pd.read_csv('/content/drive/MyDrive/result.csv')
#df.drop('Unnamed:0')
df

,Unnamed: 0,id,speech,display_as,party,constituency,mnis_id,date,time,colnum,...,RE,Verb,RE2,Pres,topic,politics_phase,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords
0,0,uk.org.publicwhip/debate/1979-11-27a.1096.7,Is the Minister aware that in the last two mon...,Robert Maclennan,Labour,Caithness and Sutherland,578.0,1979-11-27,NaN,1096.0,...,"[('shells', 'vessels', 'Is'), ('they', 'last y...","[been, have, alarmed, landed, entered, appear,...","[('shells', 'vessels', 'Is'), ('they', 'last y...","['in the last two months', 'from my constituen...",NaN,"[[], [], []]",0,8.0,0.1397,"industry, company, make, market, british, busi..."
1,1,uk.org.publicwhip/debate/1979-11-27a.1136.2,Is not the right hon. Gentleman contradicting ...,Roger Moate,Conservative,Faversham,922.0,1979-11-27,16:40:00,1136.0,...,"[('', '', 'Is'), ('Gentleman', 'himself', 'con...","[contradicting, envisages, described, told, co...","[('', '', 'Is'), ('Gentleman', 'himself', 'con...",['on what he has described as the good routes'...,NaN,"[[], [], [], [], [], []]",1,0.0,0.1437,"local, service, area, authority, constituency,..."
2,2,uk.org.publicwhip/debate/1979-11-29a.1477.7,Does the right hon. Gentleman recall that he s...,James Lamond,Labour,Oldham East,894.0,1979-11-29,NaN,1477.0,...,"[('right hon', '', 'Does'), ('colour', 'Britis...","[Does, said, fishing, regarded, reconcile, pre...","[('right hon', '', 'Does'), ('colour', 'Britis...","['for immigrant votes', 'as equal', 'of colour...",NaN,"[[], [], [], []]",2,3.0,0.1612,"country, people, government, political, agree,..."
3,3,uk.org.publicwhip/debate/1979-11-12a.1080.3,"Question put, That the amendment be made:",Unknown,NaN,NaN,NaN,1979-11-12,00:30:00,1080.0,...,"[('', 'amendment', 'put')]","[put, made]","[('', 'amendment', 'put')]",[],NaN,[[]],3,5.0,0.1214,"bill, make, order, amendment, clause, power, l..."
4,4,uk.org.publicwhip/debate/1979-11-29a.1470.2,At a most recent census in August 1979 the Uni...,Alick Buchanan-Smith,Conservative,North Angus and Mearns,727.0,1979-11-29,NaN,1470.0,...,"[('breeding herd', 'pig', 'was')]",[],"[('breeding herd', 'pig', 'was')]","['At a most recent census in August 1979', 'in...",NaN,[[]],4,4.0,0.1172,"case, public, law, police, issue, individual, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,uk.org.publicwhip/debate/2021-02-24a.1024.2,"Motion made, and Question put forthwith (Stand...",Unknown,NaN,NaN,NaN,2021-02-24,18:56:00,1024.0,...,"[('Question', '83A(7', '\n '), ('following pro...","[made, put, Standing, following, apply, supple...","[('Question', '83A(7', '\n '), ('following pro...",['to the Telecommunications Infrastructure ( L...,1.0,"[[], ['Programme ', 'Programme ']]",9995,5.0,0.2410,"bill, make, order, amendment, clause, power, l..."
9996,9996,uk.org.publicwhip/debate/2021-04-28c.361.4,I have regular discussions with ministerial co...,David Duguid,Conservative,Banff and Buchan,4606.0,2021-04-28,NaN,361.0,...,"[('COP26', 'Scotland', 'have'), ('that', 'Unit...","[have, offers, devolved, brings, devolved, sup...","[('COP26', 'Scotland', 'have'), ('that', 'Unit...","['with ministerial colleagues', 'as well as', ...",NaN,"[[], []]",9996,3.0,0.1930,"country, people, government, political, agree,..."
9997,9997,uk.org.publicwhip/debate/2021-02-22b.655.7,I am really grateful to my hon. Friend for his...,Boris Johnson,Conservative,Uxbridge and South Ruislip,1423.0,2021-02-22,15:51:00,655.0,...,"[('I', 'hon', 'am'), ('', 'question', 'Friend'...","[support, suffered, distributed, put, focus, f...","[('I', 'hon', 'am'), ('', 'question', 'Friend'...","['to my hon', 'for his question', 'to their le...",NaN,"[[], [], [], [], [], []]",9997,9.0,0.1381,"work, important, friend, support, ensure, make..."
9998,9998,uk.org.publicwhip/debate/2021-02-04b.1186.0,I welcome this debate and thank my hon. Friend...,Robert Halfon,Conservative,Harlow,3985.0,2021-02-04,15:45:00,1186.0,...,"[('I', 'hon', 'welcome'), ('', 'town f

In [6]:
data = df.speech.values.tolist()

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
#content 变成了列表，然后对列表进行一个处理
data_words = list(sent_to_words(data))

print(data_words[:1])

[['is', 'the', 'minister', 'aware', 'that', 'in', 'the', 'last', 'two', 'months', 'there', 'have', 'been', 'episodes', 'which', 'have', 'alarmed', 'fishermen', 'from', 'my', 'constituency', 'because', 'shells', 'have', 'landed', 'within', 'yards', 'of', 'their', 'vessels', 'is', 'he', 'aware', 'that', 'arrangements', 'entered', 'into', 'last', 'year', 'appear', 'not', 'to', 'be', 'working', 'as', 'they', 'might', 'will', 'he', 'examine', 'this', 'urgently']]


In [7]:
#创建bigram 和trigram 模型，这个可以说明哪些词经常在一起出现使用
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
  
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)

In [8]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

#按顺序调用函数
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN'])

print(data_lemmatized[:1])

[['month', 'episode', 'fisherman', 'constituency', 'shell', 'yard', 'vessel', 'arrangement', 'year']]


In [9]:
#创建LDA model
#LDA主题模型的两个主要输入是字典（id2word）和语料库
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1)]]


In [10]:
def trainmodel(topic):
  ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=topic, id2word=id2word)
  return ldamallet

In [11]:
from gensim.models.wrappers import ldamallet
# Show Topics ---some error with this 
ldamallet= trainmodel(6)
pprint(ldamallet.show_topics(num_words=10))
#output files
ldamallet.save("/content/drive/MyDrive/topic6/mallet_model.model")
new_model = gensim.models.ldamodel.LdaModel.load("/content/drive/MyDrive/topic6/mallet_model.model")

# # Compute Coherence Score
# coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
# coherence_ldamallet = coherence_model_ldamallet.get_coherence()
# print('\nCoherence Score: ', coherence_ldamallet)

[(0,
  '0.042*"government" + 0.033*"country" + 0.019*"state" + 0.014*"secretary" + '
  '0.014*"community" + 0.014*"today" + 0.013*"people" + 0.011*"statement" + '
  '0.011*"action" + 0.010*"decision"'),
 (1,
  '0.038*"people" + 0.033*"service" + 0.026*"child" + 0.021*"year" + '
  '0.019*"school" + 0.016*"area" + 0.014*"problem" + 0.014*"authority" + '
  '0.013*"number" + 0.013*"health"'),
 (2,
  '0.027*"industry" + 0.021*"business" + 0.020*"policy" + 0.019*"company" + '
  '0.013*"investment" + 0.013*"year" + 0.012*"government" + 0.012*"economy" + '
  '0.012*"market" + 0.012*"problem"'),
 (3,
  '0.131*"member" + 0.087*"friend" + 0.036*"point" + 0.034*"debate" + '
  '0.033*"question" + 0.030*"house" + 0.025*"issue" + 0.024*"matter" + '
  '0.019*"time" + 0.015*"parliament"'),
 (4,
  '0.081*"government" + 0.044*"people" + 0.042*"year" + 0.022*"work" + '
  '0.017*"time" + 0.017*"money" + 0.017*"tax" + 0.016*"benefit" + '
  '0.015*"labour" + 0.015*"rate"'),
 (5,
  '0.041*"bill" + 0.029*"case

In [12]:
from gensim.models.ldamodel import LdaModel
def convertldaMalletToldaGen(mallet_model):
    model_gensim = LdaModel(
        id2word=mallet_model.id2word, num_topics=mallet_model.num_topics,
        alpha=mallet_model.alpha) # original function has 'eta=0' argument
    model_gensim.state.sstats[...] = mallet_model.wordtopics
    model_gensim.sync_state()
    return model_gensim

ldagensim = convertldaMalletToldaGen(ldamallet)

In [13]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(ldagensim, corpus, id2word)
vis

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.140721  0.057493       1        1  17.447366
4      0.164060 -0.116361       2        1  17.104114
2      0.174427  0.039633       3        1  17.095163
5     -0.183583  0.309712       4        1  16.548113
0      0.015098 -0.089966       5        1  16.319188
3     -0.310723 -0.200511       6        1  15.486056, topic_info=            Term         Freq        Total Category  logprob  loglift
123       member  5714.000000  5714.000000  Default  30.0000  30.0000
25        friend  3921.000000  3921.000000  Default  29.0000  29.0000
44    government  6441.000000  6441.000000  Default  28.0000  28.0000
82        people  4590.000000  4590.000000  Default  27.0000  27.0000
235         bill  1933.000000  1933.000000  Default  26.0000  26.0000
..           ...          ...          ...      ...      ...      ...
170         hope   308.750016   606.607053   Topic6  -4.9881   1.1899
177  opportunity   349.000916   888.036896   Topic6  -4.8655   0.9313
440         fact   326.000396   932.722080   Topic6  -4.9337   0.8140
53          part   301.083169  1342.332525   Topic6  -5.0132   0.3705
70      argument   254.123774   550.143415   Topic6  -5.1828   1.0929

[346 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
1981      1  0.998700  access
226       4  0.993486     act
226       5  0.003205     act
424       4  0.209353  action
424       5  0.789216  action
...     ...       ...     ...
820       3  0.483259   world
820       5  0.515072   world
8         1  0.285297    year
8         2  0.545573    year
8         3  0.169166    year

[459 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 5, 3, 6, 1, 4])

In [14]:
pyLDAvis.save_html(vis, '/content/drive/MyDrive/topic6/ldamallet.html')

In [15]:
ldamallet.print_topics(num_words=20)

[(0,
  '0.042*"government" + 0.033*"country" + 0.019*"state" + 0.014*"secretary" + 0.014*"community" + 0.014*"today" + 0.013*"people" + 0.011*"statement" + 0.011*"action" + 0.010*"decision" + 0.010*"time" + 0.010*"proposal" + 0.010*"force" + 0.010*"agreement" + 0.009*"minister" + 0.009*"situation" + 0.009*"part" + 0.009*"woman" + 0.008*"process" + 0.008*"man"'),
 (1,
  '0.038*"people" + 0.033*"service" + 0.026*"child" + 0.021*"year" + 0.019*"school" + 0.016*"area" + 0.014*"problem" + 0.014*"authority" + 0.013*"number" + 0.013*"health" + 0.012*"education" + 0.011*"home" + 0.011*"care" + 0.010*"time" + 0.010*"system" + 0.010*"family" + 0.009*"housing" + 0.008*"support" + 0.008*"resource" + 0.007*"community"'),
 (2,
  '0.027*"industry" + 0.021*"business" + 0.020*"policy" + 0.019*"company" + 0.013*"investment" + 0.013*"year" + 0.012*"government" + 0.012*"economy" + 0.012*"market" + 0.012*"problem" + 0.011*"development" + 0.010*"interest" + 0.010*"country" + 0.009*"trade" + 0.009*"area" + 0

In [20]:
y = ldamallet.print_topics(num_words=50)
modified_format = []
for i in y:
  tmp = []
  tmp.append(i[0])
  tmp.append(i[1].split('+'))
  modified_format.append(tmp)
print(modified_format)
topic_keywords = []
for i in modified_format:
  for j in i[1]:
    topic_keywords_weight=[]
    topic_keywords_weight.append(i[0])
    y = j.split('*')
    topic_keywords_weight.append(float(y[0]))
    topic_keywords_weight.append(y[1].replace('"', ''))
    topic_keywords.append(topic_keywords_weight)

topic_keywords_weight=pd.DataFrame(topic_keywords, columns=['topic', 'fre','keywords'])    
topic_keywords_weight 
data = topic_keywords_weight.infer_objects()
data.to_csv('/content/drive/MyDrive/topic6/keywords_weight_50.csv')

[[0, ['0.042*"government" ', ' 0.033*"country" ', ' 0.019*"state" ', ' 0.014*"secretary" ', ' 0.014*"community" ', ' 0.014*"today" ', ' 0.013*"people" ', ' 0.011*"statement" ', ' 0.011*"action" ', ' 0.010*"decision" ', ' 0.010*"time" ', ' 0.010*"proposal" ', ' 0.010*"force" ', ' 0.010*"agreement" ', ' 0.009*"minister" ', ' 0.009*"situation" ', ' 0.009*"part" ', ' 0.009*"woman" ', ' 0.008*"process" ', ' 0.008*"man" ', ' 0.007*"world" ', ' 0.007*"defence" ', ' 0.007*"day" ', ' 0.007*"commitment" ', ' 0.007*"place" ', ' 0.007*"role" ', ' 0.007*"party" ', ' 0.007*"approach" ', ' 0.006*"report" ', ' 0.006*"security" ', ' 0.006*"issue" ', ' 0.006*"policy" ', ' 0.006*"nation" ', ' 0.006*"hand" ', ' 0.006*"event" ', ' 0.006*"response" ', ' 0.006*"talk" ', ' 0.005*"thing" ', ' 0.005*"end" ', ' 0.005*"responsibility" ', ' 0.005*"organisation" ', ' 0.005*"effort" ', ' 0.005*"deal" ', ' 0.005*"basis" ', ' 0.005*"meeting" ', ' 0.005*"relationship" ', ' 0.004*"progress" ', ' 0.004*"position" ', ' 0.

In [ ]:
def format_topics_sentences(ldamodel, corpus, texts):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)

df_topic_sents_keywords = format_topics_sentences(ldamodel=ldamallet, corpus=corpus, texts=data_lemmatized)
#df_topic_sents_keywords = format_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=data_lemmatized)
# Format

df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
df_dominant_topic.to_csv('/content/drive/MyDrive/topic6/df_dominant_topic.csv')


In [22]:
df_dominant_topic

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,0.0,0.1940,"government, country, state, secretary, communi...","[month, episode, fisherman, constituency, shel..."
1,1,2.0,0.2969,"industry, business, policy, company, investmen...","[contradicting, competition, route, competitio..."
2,2,0.0,0.2186,"government, country, state, secretary, communi...","[year, fishing, vote, citizen, colour, race, c..."
3,3,3.0,0.1795,"member, friend, point, debate, question, house...","[question, amendment]"
4,4,1.0,0.1795,"people, service, child, year, school, area, pr...","[herd, gilt]"
...,...,...,...,...,...
9995,9995,5.0,0.2274,"bill, case, order, amendment, clause, power, l...","[motion, question, standing_order, provision, ..."
9996,9996,0.0,0.2504,"government, country, state, secretary, communi...","[discussion, industry, stakeholder, opportunit..."
9997,9997,1.0,0.1829,"people, service, child, year, school, area, pr...","[friend, question, support, pupil, loss, learn..."
9998,9998,2.0,0.3305,"industry, business, policy, company, investmen...","[friend, member, peterborough, government, yea..."


In [ ]:
#tsen
hm = np.array([[y for (x,y) in ldamallet[corpus[i]]] for i in range(len(corpus))])
from sklearn.manifold import TSNE
tsne = TSNE(random_state=2017, perplexity=30, early_exaggeration=120)
embedding = tsne.fit_transform(hm)
embedding = pd.DataFrame(embedding, columns=['x','y'])
embedding['hue'] = hm.argmax(axis=1)
embedding.to_csv('/content/drive/MyDrive/topic6/embedding.csv')
embedding

In [ ]:
embedding = pd.read_csv('/content/drive/MyDrive/topic6/embedding.csv')
df_dominant_topic = pd.read_csv('/content/drive/MyDrive/topic6/df_dominant_topic.csv')

In [23]:
import pickle
topic_distributions = []
length = len(corpus)
for l in range(length):
  tmp = []
  tmp.append(l)
  y = ldamallet[corpus[l]]
  for i in y:
    tmp.append(i[1])
  topic_distributions.append(tmp)
with open('/content/drive/MyDrive/topic6/topic_distribution.pkl', 'wb') as f:
  pickle.dump(topic_distributions, f)

In [1]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/result.csv')
df_dominant_topic = pd.read_csv('/content/drive/MyDrive/topic6/df_dominant_topic.csv')
embedding = pd.read_csv('/content/drive/MyDrive/topic6/embedding.csv')

In [2]:
import pickle
objects = []
with (open("/content/drive/MyDrive/topic6/topic_distribution.pkl", "rb")) as openfile:
    while True:
        try:
            objects.append(pickle.load(openfile))
        except EOFError:
            break

In [10]:
df_topic_distribution=pd.DataFrame(objects[0], columns=['doc', 'topic1', 'topic2', 'topic3','topic4', 'topic5','topic6'])


In [8]:
objects[0][0]

[0,
 0.19397363465160072,
 0.1657250470809793,
 0.1713747645951036,
 0.1581920903954802,
 0.1581920903954802,
 0.15254237288135591]

In [11]:
df_topic_distribution['x'] = embedding['x']
df_topic_distribution['y'] = embedding['y']
df_topic_distribution['ref'] = df['id']
df_topic_distribution['speech'] = df['speech']
df_topic_distribution['speaker'] = df['display_as']
df_topic_distribution['location'] = df['constituency']
df_topic_distribution['party'] =df['party']
df_topic_distribution['date'] = df['date']
import datetime
def clean_date(date):
  datem = datetime.datetime.strptime(date, "%Y-%m-%d")
  return datem.year
df_topic_distribution['clean_date'] = df_topic_distribution['date'].apply(clean_date)
df_topic_distribution

,doc,topic1,topic2,topic3,topic4,topic5,topic6,x,y,ref,speech,speaker,location,party,date,clean_date
0,0,0.193974,0.165725,0.171375,0.158192,0.158192,0.152542,-3.328907,-5.634679,uk.org.publicwhip/debate/1979-11-27a.1096.7,Is the Minister aware that in the last two mon...,Robert Maclennan,Caithness and Sutherland,Labour,1979-11-27,1979
1,1,0.130208,0.131944,0.295139,0.171875,0.131944,0.138889,39.303383,15.582671,uk.org.publicwhip/debate/1979-11-27a.1136.2,Is not the right hon. Gentleman contradicting ...,Roger Moate,Faversham,Conservative,1979-11-27,1979
2,2,0.224014,0.139785,0.155914,0.197133,0.146953,0.136201,5.558863,2.299592,uk.org.publicwhip/debate/1979-11-29a.1477.7,Does the right hon. Gentleman recall that he s...,James Lamond,Oldham East,Labour,1979-11-29,1979
3,3,0.160256,0.160256,0.160256,0.179487,0.160256,0.179487,57.768925,32.940536,uk.org.publicwhip/debate/1979-11-12a.1080.3,"Question put, That the amendment be made:",Unknown,NaN,NaN,1979-11-12,1979
4,4,0.160256,0.181624,0.177350,0.160256,0.160256,0.160256,21.314182,-34.485077,uk.org.publicwhip/debate/1979-11-29a.1470.2,At a most recent census in August 1979 the Uni...,Alick Buchanan-Smith,North Angus and Mearns,Conservative,1979-11-29,1979
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,0.128205,0.135043,0.184615,0.164103,0.157265,0.230769,-25.643750,33.125770,uk.org.publicwhip/debate/2021-02-24a.1024.2,"Motion made, and Question put forthwith (Stand...",Unknown,NaN,NaN,2021-02-24,2021
9996,9996,0.242123,0.145937,0.175788,0.152570,0.139303,0.144279,-7.649641,-6.034001,uk.org.publicwhip/debate/2021-04-28c.361.4,I have regular discussions with ministerial co...,David Duguid,Banff and Buchan,Conservative,2021-04-28,2021
9997,9997,0.164103,0.194872,0.155556,0.174359,0.179487,0.131624,30.971502,4.665428,uk.org.publicwhip/debate/2021-02-22b.655.7,I am really grateful to my hon. Friend for his...,Boris Johnson,Uxbridge and South Ruislip,Conservative,2021-02-22,2021
9998,9998,0.164158,0.189964,0.324014,0.092473,0.161290,0.068100,-38.559654,-3.922474,uk.org.publicwhip/debate/2021-02-04b.1186.0,I welcome this debate and thank my hon. Friend...,Robert Halfon,Harlow,Conservative,2021-02-04,2021


In [12]:
result= pd.concat([df_topic_distribution, df_dominant_topic], axis=1)
result.to_csv('/content/drive/MyDrive/topic6/version.csv')
result

,doc,topic1,topic2,topic3,topic4,topic5,topic6,x,y,ref,...,location,party,date,clean_date,Unnamed: 0,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,0.193974,0.165725,0.171375,0.158192,0.158192,0.152542,-3.328907,-5.634679,uk.org.publicwhip/debate/1979-11-27a.1096.7,...,Caithness and Sutherland,Labour,1979-11-27,1979,0,0,0.0,0.1940,"government, country, state, secretary, communi...","['month', 'episode', 'fisherman', 'constituenc..."
1,1,0.130208,0.131944,0.295139,0.171875,0.131944,0.138889,39.303383,15.582671,uk.org.publicwhip/debate/1979-11-27a.1136.2,...,Faversham,Conservative,1979-11-27,1979,1,1,2.0,0.2969,"industry, business, policy, company, investmen...","['contradicting', 'competition', 'route', 'com..."
2,2,0.224014,0.139785,0.155914,0.197133,0.146953,0.136201,5.558863,2.299592,uk.org.publicwhip/debate/1979-11-29a.1477.7,...,Oldham East,Labour,1979-11-29,1979,2,2,0.0,0.2186,"government, country, state, secretary, communi...","['year', 'fishing', 'vote', 'citizen', 'colour..."
3,3,0.160256,0.160256,0.160256,0.179487,0.160256,0.179487,57.768925,32.940536,uk.org.publicwhip/debate/1979-11-12a.1080.3,...,NaN,NaN,1979-11-12,1979,3,3,3.0,0.1795,"member, friend, point, debate, question, house...","['question', 'amendment']"
4,4,0.160256,0.181624,0.177350,0.160256,0.160256,0.160256,21.314182,-34.485077,uk.org.publicwhip/debate/1979-11-29a.1470.2,...,North Angus and Mearns,Conservative,1979-11-29,1979,4,4,1.0,0.1795,"people, service, child, year, school, area, pr...","['herd', 'gilt']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,0.128205,0.135043,0.184615,0.164103,0.157265,0.230769,-25.643750,33.125770,uk.org.publicwhip/debate/2021-02-24a.1024.2,...,NaN,NaN,2021-02-24,2021,9995,9995,5.0,0.2274,"bill, case, order, amendment, clause, power, l...","['motion', 'question', 'standing_order', 'prov..."
9996,9996,0.242123,0.145937,0.175788,0.152570,0.139303,0.144279,-7.649641,-6.034001,uk.org.publicwhip/debate/2021-04-28c.361.4,...,Banff and Buchan,Conservative,2021-04-28,2021,9996,9996,0.0,0.2504,"government, country, state, secretary, communi...","['discussion', 'industry', 'stakeholder', 'opp..."
9997,9997,0.164103,0.194872,0.155556,0.174359,0.179487,0.131624,30.971502,4.665428,uk.org.publicwhip/debate/2021-02-22b.655.7,...,Uxbridge and South Ruislip,Conservative,2021-02-22,2021,9997,9997,1.0,0.1829,"people, service, child, year, school, area, pr...","['friend', 'question', 'support', 'pupil', 'lo..."
9998,9998,0.164158,0.189964,0.324014,0.092473,0.161290,0.068100,-38.559654,-3.922474,uk.org.publicwhip/debate/2021-02-04b.1186.0,...,Harlow,Conservative,2021-02-04,2021,9998,9998,2.0,0.3305,"industry, business, policy, company, investmen...","['friend', 'member', 'peterborough', 'governme..."
